The goal for this piece of code is:
- Clean the data for the following datasets:
    - K2
    - APOKSAC
    - APOGEE
    - GALAH

After we have cleaned the datas is
- Use K2 model to predict the mass for APOGEE and GALAH
- Use APOKSAC model to predict mass for APOGEE

The goal is to get a table contain the following for all the data sets and use this to predict the mass of stars.
The table include parameters such as:

- Effective Temperature
- Log g
- Iron Abundance [Fe/H]
- Alpha Abundance [alpha/Fe]
- Carbon Abundance [C/Fe]
- Nitrogen Abundance [N/Fe]
- Oxygen Abundance [O/Fe]

In addition, we will extract the stars id to allow for identification

- `tmass_id` 2MASS IDs
- `gaiadr3_source_id` Gaia Id

Importing the required packages

In [1]:
from astropy.table import Table
import pandas as pd         # For table/data management

# Cleaning the GALAH Data

In [3]:
# Import the data and change into a data frame
galah = Table.read("RawData/galah_dr4_allspec_220713.fits", format = "fits")
names = [name for name in galah.colnames if len(galah[name].shape) <= 1]
galah = galah[names].to_pandas()

For Apogee Data, there are a few recommendations (this usually indicates that the data has some issues, e.g. unreliably broadening and other issues) for the flag that should be applied:
- `snr_c3_iraf > 30`
- `flag_sp == 0`
- For any element `X`, the flag `flag_X_fe == 0`

More information can be found [the GALAH Survey Website](https://www.galah-survey.org/dr3/flags/)

In [4]:
# Filter the data
# galah = galah.loc[galah["snr_c3_iraf"] > 30]
galah = galah.loc[galah["flag_sp"] == 0]
galah = galah.loc[galah["flag_fe_h"] == 0]
galah = galah.loc[galah["flag_al_fe"] == 0]
galah = galah.loc[galah["flag_c_fe"] == 0]
galah = galah.loc[galah["flag_n_fe"] == 0]
galah = galah.loc[galah["flag_o_fe"] == 0]

Now the appropriate flag has been used, we extract only the useful columns and save the data for later.

In [4]:
galah = galah[["teff", "logg", "fe_h", "al_fe", "c_fe", "n_fe", "o_fe"]]
# Rmeove all rows with null values
galah.dropna()
galah.to_csv("Data/cleaned_galah.csv", index=False)

# Cleaning APOGEE Data

In [5]:
apogee = Table.read("RawData/allStar-dr17-synspec_rev1.fits", format="fits", hdu=1)
names = [name for name in apogee.colnames if len(apogee[name].shape) <= 1]
apogee = apogee[names].to_pandas()

For APOGEE, we do something similar by making sure all the parameters are working on are good.
More information can be found [SDSS explanantion of bitmasks](https://www.sdss.org/dr13/algorithms/bitmasks/#APOGEE_ASPCAPFLAG) and [SDSS recommendations for bitmasks](https://www.sdss.org/dr13/irspec/abundances/)

The parameters
- `ASPCAPFLAG & STAR_BAD == 0` (`STAR_BAD` is the 23rd bit (starting from 0), or  `STAR_BAD = 0b 1000 0000 0000 0000 0000 0000`)
    - `STAR_BAD` is a general indication of whether there are issues with measuring surface temperature, surface gravity, and check if any elements have `GRIDEDGE_BAD` set
- `ELEMFLAG`
    - Any example of this is `FE_H_FLAG`
        - Make sure that the flag does not contain `GRIDEDGE_BAD`, `GRIDEDGE_WARN` and `CALRANGE_BAD`
    - We should just check that the flag is equal to zero to eliminate all and any problematic behaviour

In [9]:
# Filter the data
STAR_BAD = 1 << 23
apogee = apogee.loc[apogee['ASPCAPFLAG'] & STAR_BAD == 0]
apogee = apogee.loc[apogee['FE_H_FLAG'] == 0]
apogee = apogee.loc[apogee['C_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['AL_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['N_FE_FLAG'] == 0]
apogee = apogee.loc[apogee['O_FE_FLAG'] == 0]

Extract the useful column and save for later

In [6]:
apogee = apogee[["TEFF", "LOGG", "FE_H", "AL_FE", "C_FE", "N_FE", "O_FE"]]
apogee.columns = apogee.columns.str.lower()
apogee.dropna()
apogee.to_csv("ProcessedData/cleaned_apogee.csv", index=False)

# Cleaning APOKSAC 2 Data
TODO: Not sure how to do this


# Cleaning K2 Data
To do so, we need to

1. extract the mass coefficient from the data set
2. find the effective temperature by finding the data in the GALAH data set by matching the EPIC identifier
3. determine the mass
4. extract the chemical composition of stars
5. store the data for use

For the purpose of this, we will keep the mass of stars as a multiple of solar mass.

In [7]:
k2_cols = ["EPIC", "kappa-m-c"]
k2_col_width = [(0, 9), (24, 29)]

k2 = pd.read_fwf(
    "Data/k2_mass_coefficients.txt",
    skiprows=50,
    header=None,
    index_col=False,
    names = k2_cols,
    colspecs = k2_col_width
)
# k2.columns = k2_cols
k2
# According to the International Astronomical Union (IAU), effective temperature of the sun is 5772 +- 0.8K
teff_sun = 5772

To determine the mass for K2 stars, we use the following formula (from K2 Galactic Archaeology Program DR3):

$$ \frac{M}{M_\odot} = \kappa_M (\frac{T_{eff}}{T_{eff, \odot}})^{3/2} $$


In [16]:
k2ToGaia = Table.read("RawData/kepler_dr3_good.fits", format = "fits")
names = [name for name in k2ToGaia.colnames if len(k2ToGaia[name].shape) <= 1]
k2ToGaia = k2ToGaia[names].to_pandas()

In [20]:
k2ToGaia.columns # Not sure where gaia id is

Index(['kepid', 'ra_kic', 'dec_kic', 'source_id', 'random_index', 'ra', 'dec',
       'parallax', 'parallax_error', 'parallax_over_error', 'pmra',
       'pmra_error', 'pmdec', 'pmdec_error', 'astrometric_params_solved',
       'pseudocolour', 'pseudocolour_error', 'ipd_frac_multi_peak',
       'ipd_frac_odd_win', 'ruwe', 'phot_g_mean_flux_over_error',
       'phot_g_mean_mag', 'phot_bp_mean_flux_over_error', 'phot_bp_mean_mag',
       'phot_rp_mean_flux_over_error', 'phot_rp_mean_mag',
       'phot_bp_rp_excess_factor', 'bp_rp', 'radial_velocity',
       'radial_velocity_error', 'rv_nb_transits', 'rv_expected_sig_to_noise',
       'rv_renormalised_gof', 'rv_chisq_pvalue', 'phot_variable_flag', 'l',
       'b', 'in_qso_candidates', 'in_galaxy_candidates', 'non_single_star',
       'has_xp_continuous', 'has_xp_sampled', 'has_rvs',
       'has_epoch_photometry', 'has_epoch_rv', 'has_mcmc_gspphot',
       'has_mcmc_msc', 'in_andromeda_survey', 'teff_gspphot', 'logg_gspphot',
       'mh_gs